# eodhd_ez
#### A simple wrapper interface around EODHD

## Get Dataframe of Ticker with Filter


In [ ]:
import os

from dotenv import load_dotenv

from eodhd_ez import EODHDEZClient

load_dotenv()

api_key = os.environ.get('EODHD_API_KEY')
ez = EODHDEZClient(api_key)

ticker_df = ez.get_tickers_df(
	exchange_code='US',
	delisted=False,
	filters={
		"Code": [
			# "MSFT",
        ],
		"Name": [
			# "Russell"
        ],
		"Exchange": [
			"NYSE",
			"NYSE ARCA",
			"AMEX",
			"NMFQS",
			"BATS",
			"NASDAQ",
			"NYSE MKT",
			"US",
		],
		"Type": [
			"Common Stock",
			# "ETF",
			# "FUND",
		],
	}
)
ticker_df = ticker_df.dropna(subset=['Code'])
ticker_df

In [ ]:
# Save Ticker Dataframe to CSV
import os

repo_dir = os.path.dirname(os.getcwd())
assert os.path.exists(repo_dir)

data_dir = os.path.join(repo_dir, 'data')
os.makedirs(data_dir, exist_ok=True)

listings_dir = os.path.join(data_dir, 'listings')
os.makedirs(listings_dir, exist_ok=True)

file_path = os.path.join(listings_dir, 'ticker_df.csv')
		
if os.path.exists(file_path):
    os.remove(file_path)

ticker_df = ticker_df.dropna()
ticker_df.to_csv(file_path)

## Get Intraday Historical Data

In [ ]:
exchange = 'US'
symbol = 'AAPL'
ticker = f'{symbol}.{exchange}'
interval = '5m'

hist = ez.get_intraday_historical_data(ticker, interval)
hist

In [ ]:
import json
import os

from eodhd import APIClient

api_key = os.environ.get('EODHD_API_KEY')
api = APIClient(api_key)

hwkn = api.get_fundamentals_data(ticker='HWKN')

eps = hwkn['Earnings']['History']['2024-09-30']
print(eps)
print(f'epsActual: {eps['epsActual']}')
print(f'epsDifference: {eps['epsDifference']}')
print(f'surprisePercent: {eps['surprisePercent']}')

# write info to json file
with open('hwkn_fundamental_data.json', 'w') as json_file:
    json.dump(hwkn, json_file, indent=2)

In [ ]:
# refund_1d_p: Number. The last day gain/loss in percent. Useful to get top gainers, losers for the past day.
# refund_5d_p: Number. The last 5 days gain/loss in percent. Useful to get top gainers, losers for the past week.
# avgvol_1d: Number. The last day volume.
# avgvol_200d: Number. The average last 200 days volume.

# Operations
# String Operations: [‘=’, ‘match’]
# Numeric Operations: [‘=’, ‘>’, ‘<‘, ‘>=’, ‘<=’]

import os

import pandas as pd
from eodhd import APIClient

api_key = os.environ.get('EODHD_API_KEY')
api = APIClient(api_key)

filters = [
    ['exchange', '=', 'US'],
    ['refund_5d_p', '<', 100],
    ['refund_5d_p', '>=', 10],
    ['avgvol_1d', '>=', 1000000],
    ['avgvol_200d', '>=', 1000000],
    ['market_capitalization', '>=', 5000000000]
]

screener = api.stock_market_screener(
    filters=filters,
    sort='market_capitalization.desc',
    signals='wallstreet_lo',
)
screener = pd.DataFrame(screener['data'])
screener

In [ ]:
import json
import os

from eodhd import APIClient

api_key = os.environ.get('EODHD_API_KEY')
api = APIClient(api_key)

msft = api.get_fundamentals_data(ticker='MSFT')

eps = msft['Earnings']['History']['2024-09-30']
print(eps)
print(f'epsActual: {eps['epsActual']}')
print(f'epsDifference: {eps['epsDifference']}')
print(f'surprisePercent: {eps['surprisePercent']}')

# write info to json file
with open('msft_fundamental_data.json', 'w') as json_file:
    json.dump(msft, json_file, indent=2)


In [ ]:
from datetime import datetime as dt

from dateutil.relativedelta import relativedelta

ticker = 'CRDF.US'

ema20 = api.get_technical_indicator_data(
    ticker=ticker,
    function='ema',
    period=20,
    date_from=(dt.now() - relativedelta(years=1, days=20 * 2, leapdays=1)).strftime('%Y-%m-%d'),
)

ema100 = api.get_technical_indicator_data(
    ticker=ticker,
    function='ema',
    period=100,
    date_from=(dt.now() - relativedelta(years=1, days=100 * 2, leapdays=1)).strftime('%Y-%m-%d'),
)

ema9 = api.get_technical_indicator_data(
    ticker=ticker,
    function='ema',
    period=9,
    date_from=(dt.now() - relativedelta(years=1, days=9 * 2, leapdays=1)).strftime('%Y-%m-%d'),
)

atr100 = api.get_technical_indicator_data(
    ticker=ticker,
    function='atr',
    period=100,
    date_from=(dt.now() - relativedelta(years=1, days=100 * 2, leapdays=1)).strftime('%Y-%m-%d'),
)

# create single dataframe with all indicators with index of date
ema20 = pd.DataFrame(ema20).set_index('date')
ema100 = pd.DataFrame(ema100).set_index('date')
ema9 = pd.DataFrame(ema9).set_index('date')
atr100 = pd.DataFrame(atr100).set_index('date')

drop_dt = (dt.now() - relativedelta(years=1, leapdays=1))

ema20 = ema20[ema20.index >= drop_dt.strftime('%Y-%m-%d')]
ema100 = ema100[ema100.index >= drop_dt.strftime('%Y-%m-%d')]
ema9 = ema9[ema9.index >= drop_dt.strftime('%Y-%m-%d')]
atr100 = atr100[atr100.index >= drop_dt.strftime('%Y-%m-%d')]

# concat all dataframes
indicators = pd.concat([ema20, ema100, ema9, atr100], axis=1)
indicators.columns = ['ema20', 'ema100', 'ema9', 'atr100']

indicators['macdv'] = ((indicators['ema20'] - indicators['ema100']) / indicators['atr100']) * 100
indicators['macdv-signal'] = indicators['ema9']
indicators['macdv-histogram'] = indicators['macdv'] - indicators['ema9']

indicators = indicators[['macdv', 'macdv-signal', 'macdv-histogram']]
indicators


In [ ]:
month = (dt.now() - relativedelta(months=1)).strftime('%Y-%m-%d')
df = indicators[indicators.index >= month]
df

In [ ]:
import os
from datetime import datetime as dt

import pandas as pd
from dateutil.relativedelta import relativedelta
from eodhd import APIClient

api_key = os.environ.get('EODHD_API_KEY')

api = APIClient(api_key)

hist = api.get_intraday_historical_data(
    symbol='GM.US',
    interval='1h',
    from_unix_time=int((dt.now() - relativedelta(dt.now(), days=7200)).timestamp()),
    to_unix_time=int(dt.now().timestamp()),
)
hist = pd.DataFrame(hist).dropna()

# Open + High + Low + Close / 4
hist['ohlc4'] = (hist['open'] + hist['high'] + hist['low'] + hist['close']) / 4

# make new df with ohlc4 and datetime converted to date column with format YYYY-MM-DD
df = hist[['datetime', 'ohlc4']].copy()
df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d')
df.set_index('datetime', inplace=True)

In [ ]:
import numpy as np
import talib

# get ohlc4 as np array
ohlc4 = np.array(df['ohlc4'], dtype=float)

macd, signal, hist = talib.MACD(ohlc4, fastperiod=12, slowperiod=26, signalperiod=9)

rsi = talib.RSI(ohlc4, timeperiod=14)

print(macd, signal, hist)
print(rsi)